In [ ]:
!pip install langchain langchain-community rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract
!pip install bitsandbytes accelerate peft safetensors sentencepiece sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (364 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:
loader = PyPDFDirectoryLoader("/content/sample_data/Data")
docs = loader.load()

In [ ]:
# create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                          chunk_overlap=30)
chunks = splitter.split_documents(docs)

In [ ]:
len(chunks)

30890

In [ ]:
chunks[1]

Document(metadata={'source': '/content/sample_data/Data/Oxford Handbook of Clinical Surgery, 4th Edition.pdf', 'page': 2}, page_content='Published and forthcoming Oxford Handbooks\nOxford Handbook for the Foundation Programme 3e')

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get("HuggingFaceToken")

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5"
)
vectorstore = Chroma.from_documents(chunks, embeddings)

<ipython-input-9-515a2f75c38d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.3, 0.7])

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1
    return tokenizer

tokenizer = initialize_tokenizer(model_name)

model = load_quantized_model(model_name)

stop_token_ids = [0]

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

<ipython-input-12-1947164b602a>:53: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)


In [ ]:
from langchain.chains import RetrievalQA

normal_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore_retreiver
)

hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ensemble_retriever
)

In [ ]:
response1 = normal_chain.invoke("what are some important surgical practices")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
response1

{'query': 'what are some important surgical practices',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n5. Operation (acute, elective, patient preparation,  surgical explorations). \n6. Basics of the laparoscopic sugery. \n \nPractices\n\nDuring the operation \n1. Appropriate surgical techniques must be applied  \n2. Change of gloves and rescrub if necessary.\n\npractices, and the good surgeon works on the perfec tion of this technique in his daily \noperating activities.  \nThe most important task in the medical education is  to teach the problemnoriented\n\nQuestion: what are some important surgical practices\nHelpful Answer: Some important surgical practices include applying appropriate surgical techniques during the operation, changing gloves and rescrubbing if necessary, and continuously working to perfect these techniques in daily operating activities.

In [ ]:
print(response1['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

5. Operation (acute, elective, patient preparation,  surgical explorations). 
6. Basics of the laparoscopic sugery. 
 
Practices

During the operation 
1. Appropriate surgical techniques must be applied  
2. Change of gloves and rescrub if necessary.

practices, and the good surgeon works on the perfec tion of this technique in his daily 
operating activities.  
The most important task in the medical education is  to teach the problemnoriented

Question: what are some important surgical practices
Helpful Answer: Some important surgical practices include applying appropriate surgical techniques during the operation, changing gloves and rescrubbing if necessary, and continuously working to perfect these techniques in daily operating activities. Additionally, problem-oriented learning is a crucial component of medical education

In [ ]:
# start_index = response1['result'].find('Helpful Answer:') + len('Helpful Answer: ')
# helpful_answer = response1['result'][start_index:]
# print(helpful_answer)

In [ ]:
response2 = hybrid_chain.invoke("what are some important surgical practices")

In [ ]:
response2

{'query': 'what are some important surgical practices',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\npractices is obligatory and the missed practice(s) must be substituted.The absentee should \nprovide the office of the department with a certifi cate.The practices are built on each other\n\npropotionally with it we increased the number of pr actices. “The new educational topics” is \npracticenoriented. In the practices the students wi ll get acquainted with the basics of surgical\n\nthe performance of a specific task or operation nee ds multiple repetitions and practices of the \nsame basic surgical techniques. With introducing of  in vitro techniques the use of animals for\n\n5. Operation (acute, elective, patient preparation,  surgical explorations). \n6. Basics of the laparoscopic sugery. \n \nPractices\n\nDuring the operation \n1. Appropriate surgical

In [ ]:
print(response2['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

practices is obligatory and the missed practice(s) must be substituted.The absentee should 
provide the office of the department with a certifi cate.The practices are built on each other

propotionally with it we increased the number of pr actices. “The new educational topics” is 
practicenoriented. In the practices the students wi ll get acquainted with the basics of surgical

the performance of a specific task or operation nee ds multiple repetitions and practices of the 
same basic surgical techniques. With introducing of  in vitro techniques the use of animals for

5. Operation (acute, elective, patient preparation,  surgical explorations). 
6. Basics of the laparoscopic sugery. 
 
Practices

During the operation 
1. Appropriate surgical techniques must be applied  
2. Change of gloves and rescrub if necessary.

practice

In [ ]:
response3 = hybrid_chain.invoke("Risks and complications in gastero intestinal surgery")

In [ ]:
print(response3['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Risks
Potential complications of colostomy surgery include:
• excessive bleeding
• surgical wound infection
• thrombophlebitis (inflammation and blood clot to veins
in the legs)
• pneumonia

exercise is safe.
Risks
Coronary artery bypass graft surgery is major surgery
and patients may experience any of the complications
associated with major surgery. The risk of death duringKEY TERMS

Surgical  Rarely required for complications of Chagas’ disease (e.g. cardiac 
or intestinal surgery).

• May be performed under GA.
• LFTs and INR needed prior to procedure.
Risks and complications
• Perforation (of the oesophagus or of the duodenum). Median risk

Fordtran’s Gastrointestinal and Liver Disease. Ed. by
Mark Feldman, et al. Philadelphia: W.B. Saunders Com-
pany, 1997.
PERIODICALS
Aliperti, Giuseppe. “Complications Related to Diagn